In [0]:
pip install tensorview

In [0]:
import tensorflow as tf
from keras.datasets import cifar10
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import tensorview as tv

# TEST DATASET 1

In [0]:
!wget http://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/BSR/BSR_bsds500.tgz

In [0]:
import tarfile
my_tar = tarfile.open('BSR_bsds500.tgz')
my_tar.extractall('./my_folder') # specify which folder to extract to
my_tar.close()

In [0]:
import glob,os
img_dir = "my_folder/BSR/BSDS500/data/images/train" 
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)

In [0]:
len(files)

In [0]:
test_data_lr = []
test_data_hr = []

In [0]:
def random_crop(img, random_crop_size): 
    h, w = img.shape[0], img.shape[1] 
    dy, dx = random_crop_size 
    x = np.random.randint(0, w-dx + 1) 
    y = np.random.randint(0, h-dy + 1) 
    return img[y:(y+dy), x:(x+dx), :] 

# Image quality test after preprocessing

In [0]:
im = cv2.imread(files[0])
im1 = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
im1 = np.expand_dims(im1,axis = -1)
im1 = np.concatenate((im1,im1,im1),axis = -1)
im2 = tf.image.rgb_to_grayscale(im)
print(im2.shape)
im2 = tf.concat([im2,im2,im2],axis = -1)

print(im2.shape)

In [0]:
compute_psnr(im,np.asarray(im2))

In [0]:
compute_psnr(im,im1)

In [0]:
plt.imshow(im1)
plt.show()
plt.imshow(im2)
plt.show()

np.subtract(im1,np.asarray(im2))
np.equal(im1,np.asarray(im2))

In [0]:
def random_crop(img, random_crop_size): 
    h, w = img.shape[0], img.shape[1] 
    dy, dx = random_crop_size 
    x = np.random.randint(0, w-dx + 1) 
    y = np.random.randint(0, h-dy + 1) 
    return img[y:(y+dy), x:(x+dx), :] 
test_data_lr = []
test_data_hr = []
for i in range(200):
  try:
    im = cv2.imread(files[i])
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    gray = gray/255.0
    gray = np.expand_dims(gray,axis = -1)  
    img_0 = np.concatenate((gray,gray,gray),axis = -1)
    img_1 = random_crop(img_0,(256,256))
    img_2 = tf.image.random_contrast(img_1,0.2,0.21)
    test_data_hr.append(np.asarray(img_1))
    test_data_lr.append(np.asarray(img_2))
  except:
    print('cannot display')    


In [0]:
test_data_lr = []
test_data_hr = []

In [0]:
for f in files:
  im = cv2.imread(f)
  try:
    img_1 = tf.image.rgb_to_grayscale(im)
    img_1 = random_crop(np.asarray(img_1),(256,256))
    img_2 = tf.image.random_contrast(img_1,0.2,0.21) 
    img_1 = tf.concat([img_1,img_1,img_1],axis = -1)
    img_2 = tf.concat([img_2,img_2,img_2],axis = -1)
    plt.imshow(img_1)
    plt.show()
    plt.imshow(img_2)
    plt.show()   
    test_data_lr.append(np.asarray(img_2))
    test_data_hr.append(np.asarray(img_1))
  except:
    print('image cannot be displayed')   

In [0]:
np.asarray(test_data_lr).shape

# TEST DATASET 2

In [0]:
!wget http://robotics.ethz.ch/~asl-datasets/flir_17_Sept_2013/asl_eth_flir.zip

In [0]:
!unzip asl_eth_flir.zip

In [0]:
import glob,os
img_dir = "flir_17_Sept_2013/Sempach-10/8bit" 
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)

In [0]:
len(files)

In [0]:
plt.imshow(cv2.imread(files[200]))
plt.show()

In [0]:
test_data_lr = []
test_data_hr = []

In [0]:
for f in files:
  im = cv2.imread(f)
  try:
    img_1 = tf.image.resize_with_crop_or_pad(im,256,256)
    img_2 = tf.image.random_contrast(img_1,0.2,0.21) 
    plt.imshow(img_1)
    plt.show()
    plt.imshow(img_2)
    plt.show()   
    test_data_lr.append(np.asarray(img_2))
    test_data_hr.append(np.asarray(img_1))
  except:
    print('image cannot be displayed')   


 

In [0]:
np.asarray(test_data_hr).shape

# TRAIN DATASET 1

In [0]:
!wget http://images.cocodataset.org/zips/val2017.zip

In [0]:
!unzip val2017.zip

In [0]:
import glob,os
img_dir = "val2017" 
data_path = os.path.join(img_dir,'*g')
files2 = glob.glob(data_path)

In [0]:
len(files2)

# DATASET PART 1

In [0]:
data2 = []
data = []
def crop_img(img):
  img_1 = tf.image.random_crop(img,size=[256,256,3],seed = 90)
  img_2 = tf.image.random_contrast(img_1,0.2,0.21)
  img_1 = tf.image.rgb_to_grayscale(img_1)
  img_2 = tf.image.rgb_to_grayscale(img_2)
  img_1 = tf.concat([img_1,img_1,img_1],axis = -1)
  img_2 = tf.concat([img_2,img_2,img_2],axis = -1)
  return img_1,img_2
for i in range(300):
  try:
    im = cv2.imread(files2[i])
    for i in range(10):
      im2,im3 = crop_img(im)
   
      target_img = im2
      input_img = im3
      data.append(np.asarray(target_img))
      data2.append(np.asarray(input_img))    
  except:
    print('Cannot work on the image')

#DATASET PART 2

In [0]:
data2_2 = []
data_2 = []
def crop_img(img):
  img_1 = tf.image.random_crop(img,size=[256,256,3],seed = 90)
  img_2 = tf.image.random_contrast(img_1,0.2,0.21)
  img_1 = tf.image.rgb_to_grayscale(img_1)
  img_2 = tf.image.rgb_to_grayscale(img_2)
  img_1 = tf.concat([img_1,img_1,img_1],axis = -1)
  img_2 = tf.concat([img_2,img_2,img_2],axis = -1)
  return img_1,img_2
for i in range(300,900):
  try:
    im = cv2.imread(files2[i])
    for i in range(5):
      im2,im3 = crop_img(im)
      plt.imshow(im2)
      plt.show()
      plt.imshow(im3)
      plt.show()
      target_img = im2
      input_img = im3
      data_2.append(np.asarray(target_img))
      data2_2.append(np.asarray(input_img))    
  except:
    print('Cannot work on the image')

In [0]:
print(np.asarray(data2_2).shape)

#DATASET PART 3

In [0]:
data2_3 = []
data_3 = []
def crop_img(img):
  img_1 = tf.image.random_crop(img,size=[256,256,3],seed = 90)
  img_2 = tf.image.random_contrast(img_1,0.2,0.21)
  img_1 = tf.image.rgb_to_grayscale(img_1)
  img_2 = tf.image.rgb_to_grayscale(img_2)
  img_1 = tf.concat([img_1,img_1,img_1],axis = -1)
  img_2 = tf.concat([img_2,img_2,img_2],axis = -1)
  return img_1,img_2
for i in range(900,1500):
  try:
    im = cv2.imread(files2[i])
    for i in range(5):
      im2,im3 = crop_img(im)
      plt.imshow(im2)
      plt.show()
      plt.imshow(im3)
      plt.show()
      target_img = im2
      input_img = im3
      data_3.append(np.asarray(target_img))
      data2_3.append(np.asarray(input_img))    
  except:
    print('Cannot work on the image')

In [0]:
np.asarray(data).shape

# DATASET PART 4

In [0]:
data2_4 = []
data_4 = []
def crop_img(img):
  img_1 = tf.image.random_crop(img,size=[256,256,3],seed = 90)
  img_2 = tf.image.random_contrast(img_1,0.2,0.21)
  img_1 = tf.image.rgb_to_grayscale(img_1)
  img_2 = tf.image.rgb_to_grayscale(img_2)
  img_1 = tf.concat([img_1,img_1,img_1],axis = -1)
  img_2 = tf.concat([img_2,img_2,img_2],axis = -1)
  return img_1,img_2
for i in range(1500,2100):
  try:
    im = cv2.imread(files2[i])
    for i in range(5):
      im2,im3 = crop_img(im)
      plt.imshow(im2)
      plt.show()
      plt.imshow(im3)
      plt.show()
      target_img = im2
      input_img = im3
      data_4.append(np.asarray(target_img))
      data2_4.append(np.asarray(input_img))    
  except:
    print('Cannot work on the image')

In [0]:
data2_5 = []
data_5 = []
def crop_img(img):
  img_1 = tf.image.random_crop(img,size=[256,256,3],seed = 90)
  img_2 = tf.image.random_contrast(img_1,0.2,0.21)
  img_1 = tf.image.rgb_to_grayscale(img_1)
  img_2 = tf.image.rgb_to_grayscale(img_2)
  img_1 = tf.concat([img_1,img_1,img_1],axis = -1)
  img_2 = tf.concat([img_2,img_2,img_2],axis = -1)
  return img_1,img_2
for i in range(2100,2700):
  try:
    im = cv2.imread(files2[i])
    for i in range(5):
      im2,im3 = crop_img(im)
      plt.imshow(im2)
      plt.show()
      plt.imshow(im3)
      plt.show()
      target_img = im2
      input_img = im3
      data_5.append(np.asarray(target_img))
      data2_5.append(np.asarray(input_img))    
  except:
    print('Cannot work on the image')

In [0]:
data2_6 = []
data_6 = []
def crop_img(img):
  img_1 = tf.image.random_crop(img,size=[256,256,3],seed = 90)
  img_2 = tf.image.random_contrast(img_1,0.2,0.21)
  img_1 = tf.image.rgb_to_grayscale(img_1)
  img_2 = tf.image.rgb_to_grayscale(img_2)
  img_1 = tf.concat([img_1,img_1,img_1],axis = -1)
  img_2 = tf.concat([img_2,img_2,img_2],axis = -1)
  return img_1,img_2
for i in range(2700,3300):
  try:
    im = cv2.imread(files2[i])
    for i in range(5):
      im2,im3 = crop_img(im)
      plt.imshow(im2)
      plt.show()
      plt.imshow(im3)
      plt.show()
      target_img = im2
      input_img = im3
      data_6.append(np.asarray(target_img))
      data2_6.append(np.asarray(input_img))    
  except:
    print('Cannot work on the image')

In [0]:
np.asarray(data_6).shape

In [0]:
data2.extend(data2_2)
data.extend(data_2)
data2.extend(data2_3)
data.extend(data_3)
data2.extend(data2_4)
data.extend(data_4)

In [0]:
data2.extend(data2_5)
data.extend(data_5)

In [0]:
data2.extend(data2_6)
data.extend(data_6)

In [0]:
np.asarray(data2).shape


In [0]:
def gen_sub(filters = 64):
  input_img = tf.keras.Input(shape=(256,256,3))
  input_image = tf.keras.layers.Conv2D(filters=64,kernel_size=(4,4),strides=2,padding = 'same')(input_img)
  c1 = tf.keras.layers.ReLU()(input_image)
  p1 = c1
  c1 = tf.keras.layers.Conv2D(filters=filters,kernel_size=(3,3),strides=1,padding = 'same')(c1)
  c1 = tf.keras.layers.BatchNormalization()(c1)
  c2 = tf.keras.layers.ReLU()(c1)
  c2 = tf.keras.layers.Conv2D(filters=filters,kernel_size=(3,3),strides=1,padding = 'same')(c2)
  c2 = tf.keras.layers.BatchNormalization()(c2)
  c3 = tf.keras.layers.ReLU()(c2)  
  u1 = tf.concat([c3,p1],axis = -1)
  u2 = tf.keras.layers.Conv2D(filters=filters,kernel_size=(1,1),strides=1)(u1)
  u3 = tf.keras.layers.Conv2DTranspose(filters = 3,kernel_size=(4,4),strides = 2,padding = 'same')(u2)
  u4 = tf.keras.activations.tanh(u3)
  gen_sub = tf.keras.Model(input_img,u4)
  return gen_sub

In [0]:
def wasserstein_loss(y_true, y_pred):
    print(y_true.shape)
    print(y_pred.shape)    
    return -tf.math.reduce_mean(y_true * y_pred)

In [0]:
def disc_sub(filters = 64):
  input_img = tf.keras.Input(shape = (256,256,6))
  c1 = tf.keras.layers.Conv2D(filters = filters,kernel_size=(4,4),strides=2,padding='same')(input_img)
  c2 = tf.keras.layers.LeakyReLU()(c1)
  c2 = tf.keras.layers.Conv2D(filters = filters*2,kernel_size=(4,4),strides=2,padding='same')(c2)
  c2 = tf.keras.layers.BatchNormalization()(c2)
  c3 = tf.keras.layers.LeakyReLU()(c2)
  c3 = tf.keras.layers.Conv2D(filters = filters*4,kernel_size=(4,4),strides=2,padding='same')(c3)
  c3 = tf.keras.layers.BatchNormalization()(c3)
  c4 = tf.keras.layers.LeakyReLU()(c3)
  c4 = tf.keras.layers.Conv2D(filters = filters*8,kernel_size=(4,4),strides=1,padding='same')(c4)
  c4 = tf.keras.layers.BatchNormalization()(c4)
  c5 = tf.keras.layers.LeakyReLU()(c4) 
  c6 = tf.keras.layers.Conv2D(filters = 1,kernel_size=(4,4),strides=1,padding='same')(c5)
  c7 = tf.keras.activations.sigmoid(c6)
  disc = tf.keras.Model(input_img,c7)
  return disc

In [0]:
disc_sub = disc_sub()
disc_sub.compile(loss=wasserstein_loss,optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002,beta_1 = 0.5),metrics=['accuracy'])

In [0]:
genet = gen_sub()
frozen = tf.keras.Model(disc_sub.inputs, disc_sub.outputs)
frozen.trainable = False


In [0]:

noise = tf.keras.Input(shape=(256,256,3))
image = genet(noise)
inp_ = tf.concat([noise,image],axis = -1)
logit = frozen(inp_)
wgan = tf.keras.Model(noise, logit)
wgan.compile(loss=wasserstein_loss,optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002,beta_1 = 0.5),metrics=['accuracy'])


In [0]:
x_train = np.asarray(data[:16384])
y_train = np.asarray(data2[:16384])

In [0]:
y_train.shape

In [0]:
tv_plot = tv.train.PlotMetrics(columns=2, wait_num=5)
batch_size = 32
n_critic = x_train.shape[0]//batch_size

In [0]:
n_critic = x_train.shape[0]//batch_size
clip_value = 0.01
epochs = 10
for epoch in range(epochs):
  for i in range(n_critic):
    batch_image = x_train[i*batch_size:(i+1)*batch_size]
    batch_noise = y_train[i*batch_size:(i+1)*batch_size]
    batch_gen_image = genet.predict(batch_noise)

    image_noise = np.concatenate((batch_image,batch_noise),axis =-1)
    gen_noise = np.concatenate((batch_gen_image,batch_noise),axis =-1)

    d_loss_real = disc_sub.train_on_batch(image_noise,-np.ones((batch_size,1)))
    d_loss_fake = disc_sub.train_on_batch(gen_noise,np.ones((batch_size,1)))
    d_loss = 0.5 * np.add(d_loss_real,d_loss_fake)
    for l in disc_sub.layers:
      weights = l.get_weights()
      weights = [np.clip(w,-clip_value,clip_value) for w in weights]
      l.set_weights(weights)
  g_loss = wgan.train_on_batch(batch_noise,-np.ones((batch_size,1)))
  tv_plot.update({'D_loss':d_loss[0],'D_binary_acc':d_loss[1],
                  'G_loss':g_loss[0],'G_binary_acc':g_loss[1]})
  tv_plot.draw()
tv_plot.visual()
tv_plot.visual(name = 'model_visual_gif',gif = True)      



#PSNR WITH DATATYPE FLOAT64

In [0]:
import math
def compute_psnr(img1, img2):
  img1 = img1.astype(np.float64) / 255.
  img2 = img2.astype(np.float64) / 255.
  mse = np.mean((img1 - img2) ** 2)
  if mse == 0:
    return "Same Image"
  return 10 * math.log10(1. / mse)
def psnr_test(test_LR,test_HR):
  image_ = test_LR
  image_gen_ = genet.predict(image_)
  
  psnr_result = compute_psnr(np.asarray(image_gen_),test_HR)
  return psnr_result

In [0]:
psnr_test(np.asarray(test_data_lr),np.asarray(test_data_hr))

In [0]:
import math
def psnr(img1, img2):
    mse = np.mean( (img1 - img2) ** 2 )
    if mse == 0:
      print("same image")
      return
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

In [0]:
def psnr_test(test_LR,test_HR):#didn't use a testing dataset
  image_b = test_LR
  image_gen_a = genet.predict(image_b)
  
  psnr_result = psnr(np.asarray(image_gen_a),test_HR)
  return psnr_result


In [0]:
psnr_test(np.asarray(test_data_lr[:256]),np.asarray(test_data_hr[:256]))

In [0]:
a

#DISCARDED CODE


In [0]:

def convert(img):
  h,w,c = img.shape
  if h == w:
    img_new = cv2.resize(img, (256, 256), cv2.INTER_AREA)
  if h > w:
    dif = h
  else:
    dif = w
  x_pos = int((dif - w)/2.)
  y_pos = int((dif - h)/2.)
  mask = np.zeros((dif, dif, c), dtype=img.dtype)
  mask[y_pos:y_pos+h, x_pos:x_pos+w, :] = img[:h, :w, :]
  img_new = cv2.resize(mask, (256, 256), cv2.INTER_AREA)  
  return img_new

            

In [0]:
def convert_gray(img):
  h,w = img.shape
  if h == w:
    img_new = cv2.resize(img, (256, 256), cv2.INTER_AREA)
  if h > w:
    dif = h
  else:
    dif = w
  x_pos = int((dif - w)/2.)
  y_pos = int((dif - h)/2.)
  mask = np.zeros((dif, dif), dtype=img.dtype)
  mask[y_pos:y_pos+h, x_pos:x_pos+w] = img[:h, :w]
  img_new = cv2.resize(mask, (256, 256), cv2.INTER_AREA)  
  return img_new

In [0]:
for f in files:
  im = cv2.imread(f)
  try:
    img_1 = tf.image.resize_with_crop_or_pad(im,256,256)
    img_1 = tf.image.rgb_to_grayscale(img_1)
    img_2 = tf.image.random_contrast(img_1,0.2,0.21) 
    img_1 = tf.concat([img_1,img_1,img_1],axis = -1)
    img_2 = tf.concat([img_2,img_2,img_2],axis = -1)
    plt.imshow(img_1)
    plt.show()
    plt.imshow(img_2)
    plt.show()   
    test_data_lr.append(np.asarray(img_2))
    test_data_hr.append(np.asarray(img_1))
  except:
    print('image cannot be displayed')   


In [0]:
def random_crop(img, random_crop_size): 
    h, w = img.shape[0], img.shape[1] 
    dy, dx = random_crop_size 
    x = np.random.randint(0, w-dx + 1) 
    y = np.random.randint(0, h-dy + 1) 
    return img[y:(y+dy), x:(x+dx), :] 

In [0]:
data2 = []
data = []
for i in range(2500):

  try:
    im = cv2.imread(files2[i])
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    gray = gray/255.0
    gray = np.expand_dims(gray,axis = -1)  
    for i in range(7):
      img_0 = np.concatenate((gray,gray,gray),axis = -1)
      img_1 = random_crop(img_0,(128,128))
      img_2 = tf.image.random_contrast(img_1,0.2,0.21)
      data.append(np.asarray(img_1))
      data2.append(np.asarray(img_2))
  except:
    print('cannot display')    
